In [18]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, explained_variance_score

In [25]:
#Load the DataSet 
column_names = ['Voltage','Output Air Temperature']
HD_df = pd.read_csv(os.path.abspath('hairdryer.csv'),names=column_names)
HD_df.head()

,Voltage,Output Air Temperature
0,6.41,4.7661
1,3.41,4.7637
2,6.41,4.8394
3,6.41,5.0030
4,6.41,5.0176


In [65]:
#Create the x and y variables
x = HD_df['Voltage'].values.reshape(-1, 1)
y = HD_df['Output Air Temperature'].values.reshape(-1, 1)

#Split the variables into train and test sets
seed = 7
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=seed)

#Squeeze y_train to correct the dimensions
y_train = np.squeeze(y_train)

In [75]:
#Cluster the input-output space
cl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=3)
clust_centers, part_matrix, _ = cl.cluster(method='fcm')

In [72]:
#Estimate membership functions parameters
ae = AntecedentEstimator(X_train, part_matrix)
antecedent_params = ae.determineMF()

In [73]:
#Estimate consequent parameters
ce = ConsequentEstimator(X_train, y_train, part_matrix)
conseq_params = ce.suglms()

In [69]:
# Print out the inputs
print(f"Antecedent Parameters: {antecedent_params}")  # Check if this matches expected inputs
print(f"Consequent Parameters: {conseq_params}")      # Check the length and structure
print(f"Column Names: {column_names}")                # Ensure this aligns with antecedent and consequent

# Also, check the lengths
print(f"Number of Antecedent Sets: {len(antecedent_params)}")
print(f"Number of Consequent Sets: {len(conseq_params)}")
print(f"Number of Column Names: {len(column_names)}")

Antecedent Parameters: [('gauss', array([6.97418931, 1.9987648 ])), ('gauss', array([2.78762261, 2.15750951]))]
Consequent Parameters: [[ 0.05472914  4.67967836]
 [-0.03944408  4.90710198]]
Column Names: ['Voltage', 'Output Air Temperature']
Number of Antecedent Sets: 2
Number of Consequent Sets: 2
Number of Column Names: 2


In [74]:
#Build first-order Takagi-Sugeno model
modbuilder = SugenoFISBuilder(antecedent_params, conseq_params, column_names, save_simpful_code=False)
model = modbuilder.get_model()

 * Detected 3 rules / clusters


IndexError: list index out of range

In [60]:
#Get model predictions
modtester = SugenoFISTester(model, X_test, column_names)
y_pred = modtester.predict()[0]

NameError: name 'model' is not defined

In [ ]:
# %% Compute regression metrics
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: {:.3f}".format(mse))
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error: {:.1f}%".format(mape*100))
exp_var = explained_variance_score(y_test, y_pred)
print("Explained Variance Score: {:.3f}".format(exp_var))